In [2]:
%load_ext autoreload

In [26]:
%autoreload
import os
import importlib
import itertools
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
from torchvision.utils import save_image
import torchvision.transforms as transforms

import MNISTM_dataset
from usps_dataset import USPS
from blocks import ResidualBlock, weights_initialization

In [28]:
use_cuda = torch.cuda.is_available()
root = 'data/'
batch_size = 32

latent_dimension = 10
img_channels = 1 # fixed for all
img_size = 32
# lr_C = 2e-4
learning_rate = 2e-4
beta_1 = 0.5 # ?
beta_2 = 0.999 # ?

## Define two datasets (MNIST, MNIST_M, SVHN):

In [29]:
os.makedirs('data/', exist_ok=True)

In [30]:
trans = transforms.Compose([transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),
                                                                                        (0.5, 0.5, 0.5))])

# # 
# transforms.ToPILImage(mode='F'), 
usps_trans = transforms.Compose([transforms.Resize(img_size),
                                 transforms.ToTensor(), 
                                 transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

In [31]:
training_set_original = dset.MNIST(root=root, train=True, transform=trans, download=False)
test_set_original = dset.MNIST(root=root, train=False, transform=trans, download=False)

In [32]:
training_set_m = MNISTM_dataset.MNISTM(root=root, train=True, transform=trans, download=False)
test_set_m = MNISTM_dataset.MNISTM(root=root, train=False, transform=trans, download=False)

In [33]:
# training_set_svhn = dset.SVHN(root=root, split='extra', transform=trans, download=False)
# test_set_svhn = dset.SVHN(root=root, split='test', transform=trans, download=False)

In [34]:
training_usps = USPS(root=root, train=True,transform=usps_trans, download=False)

In [35]:
dataloader_original = torch.utils.data.DataLoader(training_set_original, batch_size=batch_size, 
                                                  shuffle=True, num_workers=6, pin_memory=True, drop_last=True)
dataloader_MNISTM = torch.utils.data.DataLoader(training_set_m, batch_size=batch_size, 
                                                shuffle=True, num_workers=6, pin_memory=True)
# dataloader_svhn = torch.utils.data.DataLoader(training_set_svhn, batch_size=batch_size, 
#                                                 shuffle=True, num_workers=6, pin_memory=True, drop_last=True)

usps_data_loader = torch.utils.data.DataLoader(
        dataset=training_usps,
        batch_size=batch_size, shuffle=True, num_workers=6, pin_memory=True, drop_last=True)

dataloader_original_test = torch.utils.data.DataLoader(test_set_original, batch_size=batch_size, 
                                                  shuffle=False, num_workers=6, pin_memory=True)
dataloader_MNISTM_test = torch.utils.data.DataLoader(test_set_m, batch_size=batch_size, 
                                                shuffle=False, num_workers=6, pin_memory=True)
# dataloader_svhn_test = torch.utils.data.DataLoader(test_set_svhn, batch_size=batch_size, 
#                                                 shuffle=False, num_workers=6, pin_memory=True)

## Define generator, discriminator and classifier architectures


In [119]:
class Generator(nn.Module):
    def __init__(self, latent_dimension=10, img_channels=1, img_size=32):
        super().__init__()
        
        self.fc = nn.Linear(latent_dimension, img_channels*img_size**2)
        self.l1 = nn.Sequential(nn.Conv2d(img_channels*2, 64, 3, 1, 1), nn.ReLU(inplace=True))
        
        residual_blocks = []
        
        for i in range(6):
            residual_blocks.append(ResidualBlock())
        self.residual_blocks = nn.Sequential(*residual_blocks)
        
        self.l2 = nn.Sequential(nn.Conv2d(64, img_channels, 3, 1, 1), nn.Tanh())
         
    def forward(self, img, z):
        transformed_noise = self.fc(z).view(*img.shape)
        transformed_input = torch.cat((img, transformed_noise ), 1)
        out = self.l1(transformed_input)
        out = self.residual_blocks(out)
        img = self.l2(out)
        
        return img

In [120]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        def block(in_features, out_features, normalize=True):
            layers = [nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                     nn.LeakyReLU(0.2, inplace=True)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_features))
            return layers
        
        self.model = nn.Sequential(
            *block(img_channels, 64, normalize=False),
            torch.nn.Dropout(p=0.1, inplace=True),
            *block(64, 128),
            torch.nn.Dropout(p=0.1, inplace=True),
            *block(128,256),
            torch.nn.Dropout(p=0.1, inplace=True),
            *block(256,512),
            torch.nn.Dropout(p=0.1, inplace=True),
            nn.Conv2d(512,1,3,1,1)
        )
        
    def forward(self, img):
        out = self.model(img)
        
        return out

In [121]:
class Classifier(nn.Module):
    def __init__(self, img_size=32, number_of_classes=10):
        super().__init__()
        
        input_size = img_size // 2**4
        
        def block(in_features, out_features, normalize=True):
            layers = [nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                     nn.LeakyReLU(0.2, inplace=True)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_features))
            return layers
        
        self.model = nn.Sequential(
            *block(img_channels, 64, normalize=False),
            *block(64, 128),
            *block(128,256),
            *block(256,512)
        ) # pop the last layer from the Discriminator
        
        self.classifier = nn.Sequential(
            nn.Linear(512*input_size**2, number_of_classes),
            nn.Softmax(dim=0)
        )
        
    def forward(self, img):
        features = self.model(img)
        features = features.view(features.size(0), -1)
        labels = self.classifier(features)
        return labels

In [174]:
generator, discriminator, classifier = Generator(), Discriminator(), Classifier()

if use_cuda: generator.cuda(); discriminator.cuda(); classifier.cuda()

generator.apply(weights_initialization); discriminator.apply(weights_initialization); classifier.apply(weights_initialization); 

## Define loss function and optimizers

In [175]:
MSE_loss = torch.nn.MSELoss()
task_loss = torch.nn.CrossEntropyLoss()

lambda_gan = 1 #0.05
lambda_task = 1 # Loss weights

In [176]:
opt_G = torch.optim.Adam(itertools.chain(generator.parameters(), classifier.parameters()),
                        lr = learning_rate, betas = (beta_1, beta_2))

opt_D = torch.optim.Adam(itertools.chain(discriminator.parameters()),
                        lr = learning_rate, betas = (beta_1, beta_2))

In [177]:
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor

## GAN Training

In [178]:
patch = int(img_size / 2**4)
patch = (1, patch, patch)

In [179]:
task_performance = []
target_performance = []

In [180]:
len(dataloader_original)

1875

In [181]:
n_epochs = 200

classifier.train()

for epoch in range(n_epochs):
    for i, ((imgs_A, labels_A), (imgs_B, labels_B)) in enumerate(zip(itertools.cycle(usps_data_loader),
                                                                     dataloader_original)):
        
        real = Variable(FloatTensor(batch_size, *patch).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, *patch).fill_(0.0), requires_grad=False)

        imgs_A =  Variable(imgs_A.type(FloatTensor))#.expand(batch_size, 3, img_size, img_size) # expand if MNIST
        labels_A = Variable(labels_A.type(LongTensor).squeeze())
        imgs_B = Variable(imgs_B.type(FloatTensor))#.expand(batch_size, 3, img_size, img_size) # pointless expanding?
        
        ####################
        # Train generator
        ####################
        
        opt_G.zero_grad()
         
        # Generate noise: # used to be uniform -1 to 1 # sample from Normal instead?
        z = Variable(FloatTensor(np.random.normal(0,1, (batch_size, latent_dimension)))) 
        
        # Generate artificial images based on noise input:
        fake_B = generator(imgs_A, z)
        
        pred_fB = classifier(fake_B)
#         print(labels_A)
#         print(labels_B)
        task_loss_ = (task_loss(pred_fB, labels_A) + task_loss(classifier(imgs_A), labels_A)) / 2.
        
        g_loss = lambda_gan * MSE_loss(discriminator(fake_B), real) + lambda_task * task_loss_
        
        g_loss.backward()
        opt_G.step()
        
        ####################
        # Train discriminator
        ####################
        
        opt_D.zero_grad()
        
        real_loss = MSE_loss(discriminator(imgs_B), real)
        fake_loss = MSE_loss(discriminator(fake_B.detach()), fake)
        
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        opt_D.step()
        
        #############################
        # Evaluate target performance
        #############################
        
        acc = np.mean(np.argmax(pred_fB.data.cpu().numpy(), axis=1) == labels_A.data.cpu().numpy())
        task_performance.append(acc)
        
        pred_B = classifier(imgs_B)
        target_acc = np.mean(np.argmax(pred_B.data.cpu().numpy(), axis=1) == labels_B.numpy())
        target_performance.append(target_acc)
        if len(target_performance) > 100:
            target_performance.pop(0)
        
        batches_done = len(dataloader_original) * epoch + i
        
        if batches_done%50==0:
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] \
            [CLF acc: %3d%% (%3d%%), target_acc: %3d%% (%3d%%)]" %
            (epoch, n_epochs,
            i, len(dataloader_original),
            d_loss.data[0], g_loss.data[0],
            100*acc, 100*np.mean(task_performance),
            100*target_acc, 100*np.mean(target_performance)))
        
        interval = 600
        
        if batches_done % interval == 0:
            sample = torch.cat((imgs_A.data[:5], fake_B.data[:5], imgs_B.data[:5]), -2)
            save_image(sample, 'usps_images/%d.png' % batches_done, nrow=int(np.sqrt(batch_size)), normalize=True)        

[Epoch 0/200] [Batch 0/1875] [D loss: 0.723116] [G loss: 3.475370]             [CLF acc:  12% ( 12%), target_acc:  12% ( 12%)]
[Epoch 0/200] [Batch 50/1875] [D loss: 0.155917] [G loss: 2.866313]             [CLF acc:  81% ( 74%), target_acc:  65% ( 47%)]
[Epoch 0/200] [Batch 100/1875] [D loss: 0.169509] [G loss: 2.602521]             [CLF acc:  78% ( 77%), target_acc:  56% ( 51%)]
[Epoch 0/200] [Batch 150/1875] [D loss: 0.189241] [G loss: 2.404200]             [CLF acc:  93% ( 78%), target_acc:  62% ( 55%)]
[Epoch 0/200] [Batch 200/1875] [D loss: 0.125679] [G loss: 2.724490]             [CLF acc:  93% ( 79%), target_acc:  56% ( 57%)]
[Epoch 0/200] [Batch 250/1875] [D loss: 0.133265] [G loss: 2.721319]             [CLF acc:  90% ( 79%), target_acc:  62% ( 58%)]
[Epoch 0/200] [Batch 300/1875] [D loss: 0.156605] [G loss: 2.705920]             [CLF acc:  71% ( 78%), target_acc:  71% ( 61%)]
[Epoch 0/200] [Batch 350/1875] [D loss: 0.115290] [G loss: 3.009952]             [CLF acc:  75% ( 77

[Epoch 1/200] [Batch 1325/1875] [D loss: 0.170082] [G loss: 2.633630]             [CLF acc:  87% ( 83%), target_acc:  81% ( 87%)]
[Epoch 1/200] [Batch 1375/1875] [D loss: 0.207017] [G loss: 2.467963]             [CLF acc:  75% ( 83%), target_acc:  96% ( 87%)]
[Epoch 1/200] [Batch 1425/1875] [D loss: 0.208394] [G loss: 2.364604]             [CLF acc: 100% ( 83%), target_acc: 100% ( 89%)]
[Epoch 1/200] [Batch 1475/1875] [D loss: 0.207036] [G loss: 2.463852]             [CLF acc:  81% ( 83%), target_acc:  81% ( 89%)]
[Epoch 1/200] [Batch 1525/1875] [D loss: 0.178137] [G loss: 2.478129]             [CLF acc:  90% ( 83%), target_acc:  81% ( 89%)]
[Epoch 1/200] [Batch 1575/1875] [D loss: 0.190412] [G loss: 2.403313]             [CLF acc:  90% ( 83%), target_acc:  65% ( 89%)]
[Epoch 1/200] [Batch 1625/1875] [D loss: 0.204497] [G loss: 2.461566]             [CLF acc:  84% ( 83%), target_acc:  90% ( 88%)]
[Epoch 1/200] [Batch 1675/1875] [D loss: 0.227675] [G loss: 2.281860]             [CLF acc

[Epoch 3/200] [Batch 775/1875] [D loss: 0.160747] [G loss: 2.477669]             [CLF acc:  90% ( 86%), target_acc:  87% ( 91%)]
[Epoch 3/200] [Batch 825/1875] [D loss: 0.185498] [G loss: 2.382774]             [CLF acc:  90% ( 86%), target_acc:  96% ( 90%)]
[Epoch 3/200] [Batch 875/1875] [D loss: 0.171890] [G loss: 2.601860]             [CLF acc: 100% ( 86%), target_acc:  90% ( 90%)]
[Epoch 3/200] [Batch 925/1875] [D loss: 0.166415] [G loss: 2.558974]             [CLF acc:  93% ( 86%), target_acc: 100% ( 91%)]
[Epoch 3/200] [Batch 975/1875] [D loss: 0.191141] [G loss: 2.505960]             [CLF acc:  90% ( 86%), target_acc:  75% ( 91%)]
[Epoch 3/200] [Batch 1025/1875] [D loss: 0.185249] [G loss: 2.502291]             [CLF acc: 100% ( 86%), target_acc:  90% ( 91%)]
[Epoch 3/200] [Batch 1075/1875] [D loss: 0.167613] [G loss: 2.422622]             [CLF acc:  87% ( 86%), target_acc:  96% ( 92%)]
[Epoch 3/200] [Batch 1125/1875] [D loss: 0.176148] [G loss: 2.438097]             [CLF acc:  93

[Epoch 5/200] [Batch 225/1875] [D loss: 0.150107] [G loss: 2.488306]             [CLF acc:  87% ( 88%), target_acc:  93% ( 92%)]
[Epoch 5/200] [Batch 275/1875] [D loss: 0.184134] [G loss: 2.507007]             [CLF acc:  96% ( 88%), target_acc:  87% ( 92%)]
[Epoch 5/200] [Batch 325/1875] [D loss: 0.187520] [G loss: 2.448972]             [CLF acc:  84% ( 88%), target_acc:  93% ( 92%)]
[Epoch 5/200] [Batch 375/1875] [D loss: 0.213448] [G loss: 2.334330]             [CLF acc:  87% ( 88%), target_acc:  96% ( 92%)]
[Epoch 5/200] [Batch 425/1875] [D loss: 0.138336] [G loss: 2.634629]             [CLF acc:  87% ( 88%), target_acc:  90% ( 93%)]
[Epoch 5/200] [Batch 475/1875] [D loss: 0.213314] [G loss: 2.674721]             [CLF acc:  96% ( 88%), target_acc:  96% ( 92%)]
[Epoch 5/200] [Batch 525/1875] [D loss: 0.179171] [G loss: 2.472076]             [CLF acc: 100% ( 88%), target_acc:  87% ( 92%)]
[Epoch 5/200] [Batch 575/1875] [D loss: 0.182380] [G loss: 2.752507]             [CLF acc:  96% (

[Epoch 6/200] [Batch 1550/1875] [D loss: 0.170318] [G loss: 2.568506]             [CLF acc:  87% ( 89%), target_acc:  90% ( 92%)]
[Epoch 6/200] [Batch 1600/1875] [D loss: 0.125164] [G loss: 2.524397]             [CLF acc:  96% ( 89%), target_acc:  93% ( 92%)]
[Epoch 6/200] [Batch 1650/1875] [D loss: 0.179079] [G loss: 2.590175]             [CLF acc:  87% ( 89%), target_acc:  96% ( 93%)]
[Epoch 6/200] [Batch 1700/1875] [D loss: 0.200072] [G loss: 2.330822]             [CLF acc:  90% ( 89%), target_acc:  84% ( 93%)]
[Epoch 6/200] [Batch 1750/1875] [D loss: 0.162927] [G loss: 2.648023]             [CLF acc:  87% ( 89%), target_acc: 100% ( 93%)]
[Epoch 6/200] [Batch 1800/1875] [D loss: 0.191206] [G loss: 2.441138]             [CLF acc: 100% ( 89%), target_acc:  93% ( 93%)]
[Epoch 6/200] [Batch 1850/1875] [D loss: 0.212055] [G loss: 2.440490]             [CLF acc:  81% ( 89%), target_acc: 100% ( 94%)]
[Epoch 7/200] [Batch 25/1875] [D loss: 0.169975] [G loss: 2.568215]             [CLF acc: 

...

[Epoch 191/200] [Batch 1275/1875] [D loss: 0.117787] [G loss: 2.619178]             [CLF acc:  96% ( 95%), target_acc:  93% ( 96%)]
[Epoch 191/200] [Batch 1325/1875] [D loss: 0.094744] [G loss: 2.742649]             [CLF acc: 100% ( 95%), target_acc:  96% ( 96%)]
[Epoch 191/200] [Batch 1375/1875] [D loss: 0.118121] [G loss: 2.689527]             [CLF acc: 100% ( 95%), target_acc: 100% ( 96%)]
[Epoch 191/200] [Batch 1425/1875] [D loss: 0.126560] [G loss: 2.731215]             [CLF acc:  93% ( 95%), target_acc:  96% ( 96%)]
[Epoch 191/200] [Batch 1475/1875] [D loss: 0.093298] [G loss: 2.639911]             [CLF acc:  96% ( 95%), target_acc:  90% ( 96%)]
[Epoch 191/200] [Batch 1525/1875] [D loss: 0.104739] [G loss: 2.560543]             [CLF acc: 100% ( 95%), target_acc:  96% ( 96%)]
[Epoch 191/200] [Batch 1575/1875] [D loss: 0.127589] [G loss: 2.657593]             [CLF acc:  96% ( 95%), target_acc:  96% ( 97%)]
[Epoch 191/200] [Batch 1625/1875] [D loss: 0.095694] [G loss: 2.645752]     

[Epoch 193/200] [Batch 675/1875] [D loss: 0.118096] [G loss: 2.678410]             [CLF acc:  90% ( 95%), target_acc:  96% ( 96%)]
[Epoch 193/200] [Batch 725/1875] [D loss: 0.096608] [G loss: 2.748389]             [CLF acc: 100% ( 95%), target_acc:  96% ( 97%)]
[Epoch 193/200] [Batch 775/1875] [D loss: 0.087959] [G loss: 2.643601]             [CLF acc:  93% ( 95%), target_acc:  93% ( 96%)]
[Epoch 193/200] [Batch 825/1875] [D loss: 0.139051] [G loss: 2.573006]             [CLF acc:  90% ( 95%), target_acc: 100% ( 95%)]
[Epoch 193/200] [Batch 875/1875] [D loss: 0.106208] [G loss: 2.731059]             [CLF acc: 100% ( 95%), target_acc:  93% ( 95%)]
[Epoch 193/200] [Batch 925/1875] [D loss: 0.103047] [G loss: 2.869134]             [CLF acc:  93% ( 95%), target_acc:  93% ( 95%)]
[Epoch 193/200] [Batch 975/1875] [D loss: 0.084073] [G loss: 2.848564]             [CLF acc: 100% ( 95%), target_acc:  96% ( 96%)]
[Epoch 193/200] [Batch 1025/1875] [D loss: 0.122798] [G loss: 2.946030]            

[Epoch 195/200] [Batch 75/1875] [D loss: 0.101010] [G loss: 2.915329]             [CLF acc:  96% ( 95%), target_acc: 100% ( 95%)]
[Epoch 195/200] [Batch 125/1875] [D loss: 0.088368] [G loss: 2.937643]             [CLF acc:  93% ( 95%), target_acc:  93% ( 96%)]
[Epoch 195/200] [Batch 175/1875] [D loss: 0.120285] [G loss: 2.933326]             [CLF acc:  93% ( 95%), target_acc:  96% ( 96%)]
[Epoch 195/200] [Batch 225/1875] [D loss: 0.077005] [G loss: 2.719350]             [CLF acc:  96% ( 95%), target_acc:  96% ( 96%)]
[Epoch 195/200] [Batch 275/1875] [D loss: 0.117434] [G loss: 2.704764]             [CLF acc: 100% ( 95%), target_acc:  93% ( 96%)]
[Epoch 195/200] [Batch 325/1875] [D loss: 0.125771] [G loss: 2.815919]             [CLF acc:  93% ( 95%), target_acc: 100% ( 96%)]
[Epoch 195/200] [Batch 375/1875] [D loss: 0.143314] [G loss: 2.605849]             [CLF acc: 100% ( 95%), target_acc:  96% ( 95%)]
[Epoch 195/200] [Batch 425/1875] [D loss: 0.140732] [G loss: 2.658377]             [

[Epoch 196/200] [Batch 1350/1875] [D loss: 0.091710] [G loss: 2.781975]             [CLF acc: 100% ( 95%), target_acc:  96% ( 96%)]
[Epoch 196/200] [Batch 1400/1875] [D loss: 0.096225] [G loss: 2.753656]             [CLF acc:  93% ( 95%), target_acc:  84% ( 96%)]
[Epoch 196/200] [Batch 1450/1875] [D loss: 0.111756] [G loss: 2.729610]             [CLF acc:  96% ( 95%), target_acc:  96% ( 95%)]
[Epoch 196/200] [Batch 1500/1875] [D loss: 0.123811] [G loss: 2.767717]             [CLF acc:  96% ( 95%), target_acc:  96% ( 96%)]
[Epoch 196/200] [Batch 1550/1875] [D loss: 0.100373] [G loss: 2.851329]             [CLF acc: 100% ( 95%), target_acc:  96% ( 96%)]
[Epoch 196/200] [Batch 1600/1875] [D loss: 0.110514] [G loss: 2.850644]             [CLF acc:  96% ( 95%), target_acc: 100% ( 96%)]
[Epoch 196/200] [Batch 1650/1875] [D loss: 0.103564] [G loss: 3.020880]             [CLF acc:  93% ( 95%), target_acc:  93% ( 96%)]
[Epoch 196/200] [Batch 1700/1875] [D loss: 0.107401] [G loss: 2.766445]     

[Epoch 198/200] [Batch 750/1875] [D loss: 0.106174] [G loss: 2.778539]             [CLF acc:  96% ( 95%), target_acc:  93% ( 96%)]
[Epoch 198/200] [Batch 800/1875] [D loss: 0.101048] [G loss: 2.731143]             [CLF acc: 100% ( 95%), target_acc: 100% ( 96%)]
[Epoch 198/200] [Batch 850/1875] [D loss: 0.116283] [G loss: 2.733855]             [CLF acc:  96% ( 95%), target_acc:  96% ( 96%)]
[Epoch 198/200] [Batch 900/1875] [D loss: 0.124279] [G loss: 2.664108]             [CLF acc: 100% ( 95%), target_acc:  96% ( 97%)]
[Epoch 198/200] [Batch 950/1875] [D loss: 0.111123] [G loss: 2.685743]             [CLF acc:  96% ( 95%), target_acc: 100% ( 96%)]
[Epoch 198/200] [Batch 1000/1875] [D loss: 0.092601] [G loss: 3.008603]             [CLF acc:  84% ( 95%), target_acc:  93% ( 96%)]
[Epoch 198/200] [Batch 1050/1875] [D loss: 0.098016] [G loss: 2.637110]             [CLF acc: 100% ( 95%), target_acc: 100% ( 96%)]
[Epoch 198/200] [Batch 1100/1875] [D loss: 0.122194] [G loss: 2.663433]          

## Save/Load trained models and results

In [60]:
# torch.save(generator,'generator_250.pt')
# torch.save(discriminator,'discriminator_250.pt')
# torch.save(classifier,'classifier_g_250.pt')
generator = torch.load('generator_250.pt')
discriminator = torch.load('discriminator_250.pt')
classifier = torch.load('classifier_g_250.pt')

In [36]:
# np.save('target_performance.npy', target_performance)
# np.save('task_performance.npy', task_performance)
task_performance = np.load('task_performance.npy')
target_performance = np.load('target_performance.npy')

## Train a clean MNIST-M-only classifier

In [130]:
class SimpleClassifier(nn.Module): # non-convolutional
    def __init__(self,input_size,hidden_size, num_classes=10):
        super().__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, img):
        out = self.l1(img)
        out = self.relu(out)
        out = self.l2(out)
        out = self.softmax(out)
        return out

In [131]:
simple_classifier = SimpleClassifier(input_size=img_channels*img_size**2, hidden_size=500).cuda()
simple_classifier.apply(weights_initialization);

classifier_performance = []

lr_C = 1e-4 # 1e-4 worked
opt_C = torch.optim.Adam(itertools.chain( simple_classifier.parameters()),
                        lr = lr_C, betas = (beta_1, beta_2))

In [160]:
n_epochs = 50
simple_classifier.train()

for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader_original):
        
        imgs = Variable(imgs.view(-1,img_channels*img_size**2).type(FloatTensor))
        labels_var = Variable(labels.type(LongTensor))
        labels = labels.type(LongTensor)
        
        opt_C.zero_grad()
        
#         images.view(-1, 28*28))
#         print(imgs)
        output = simple_classifier(imgs)
        c_loss = task_loss(output, labels_var)
        c_loss.backward()
        opt_C.step()
        
        acc = np.mean(torch.max(output.data, 1)[1] == labels)
        
        classifier_performance.append(acc)
        
        print ("[Epoch %d/%d] [Batch %d/%d] [C loss: %f] \
                [CLF acc: %3d%% (%3d%%)]" %
                (epoch, n_epochs,
                i, len(dataloader_MNISTM),
                c_loss.data[0],
                100*acc, 100*np.mean(classifier_performance[-100:])))
        
        # watch out for overfit - we don't have validation accuracy here

[Epoch 0/50] [Batch 0/1875] [C loss: 2.034029]                 [CLF acc:  87% ( 94%)]
[Epoch 0/50] [Batch 1/1875] [C loss: 2.033434]                 [CLF acc:  93% ( 93%)]
[Epoch 0/50] [Batch 2/1875] [C loss: 2.034209]                 [CLF acc:  96% ( 93%)]
[Epoch 0/50] [Batch 3/1875] [C loss: 2.065620]                 [CLF acc:  93% ( 93%)]
[Epoch 0/50] [Batch 4/1875] [C loss: 2.030325]                 [CLF acc:  93% ( 93%)]
[Epoch 0/50] [Batch 5/1875] [C loss: 2.060790]                 [CLF acc:  90% ( 93%)]
[Epoch 0/50] [Batch 6/1875] [C loss: 2.028842]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 7/1875] [C loss: 2.030509]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 8/1875] [C loss: 2.093218]                 [CLF acc:  87% ( 93%)]
[Epoch 0/50] [Batch 9/1875] [C loss: 2.030056]                 [CLF acc: 100% ( 93%)]
[Epoch 0/50] [Batch 10/1875] [C loss: 2.028739]                 [CLF acc:  96% ( 93%)]
[Epoch 0/50] [Batch 11/1875] [C loss: 2.033269]      

[Epoch 0/50] [Batch 121/1875] [C loss: 2.039670]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 122/1875] [C loss: 2.042841]                 [CLF acc:  90% ( 94%)]
[Epoch 0/50] [Batch 123/1875] [C loss: 2.061589]                 [CLF acc:  90% ( 94%)]
[Epoch 0/50] [Batch 124/1875] [C loss: 2.029673]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 125/1875] [C loss: 2.061204]                 [CLF acc:  90% ( 94%)]
[Epoch 0/50] [Batch 126/1875] [C loss: 2.032249]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 127/1875] [C loss: 2.032092]                 [CLF acc:  90% ( 94%)]
[Epoch 0/50] [Batch 128/1875] [C loss: 2.031855]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 129/1875] [C loss: 2.031326]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 130/1875] [C loss: 2.031502]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 131/1875] [C loss: 2.031207]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 132/1875] [C

[Epoch 0/50] [Batch 254/1875] [C loss: 2.029450]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 255/1875] [C loss: 2.028736]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 256/1875] [C loss: 2.059366]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 257/1875] [C loss: 2.031936]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 258/1875] [C loss: 2.031665]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 259/1875] [C loss: 2.062708]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 260/1875] [C loss: 2.033610]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 261/1875] [C loss: 2.029972]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 262/1875] [C loss: 2.053926]                 [CLF acc:  90% ( 94%)]
[Epoch 0/50] [Batch 263/1875] [C loss: 2.030409]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 264/1875] [C loss: 2.090477]                 [CLF acc:  87% ( 94%)]
[Epoch 0/50] [Batch 265/1875] [C

[Epoch 0/50] [Batch 389/1875] [C loss: 2.089321]                 [CLF acc:  87% ( 94%)]
[Epoch 0/50] [Batch 390/1875] [C loss: 2.030853]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 391/1875] [C loss: 2.038412]                 [CLF acc:  90% ( 94%)]
[Epoch 0/50] [Batch 392/1875] [C loss: 2.062356]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 393/1875] [C loss: 2.029985]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 394/1875] [C loss: 2.031317]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 395/1875] [C loss: 2.028621]                 [CLF acc:  96% ( 95%)]
[Epoch 0/50] [Batch 396/1875] [C loss: 2.091472]                 [CLF acc:  84% ( 94%)]
[Epoch 0/50] [Batch 397/1875] [C loss: 2.028958]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 398/1875] [C loss: 2.030861]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 399/1875] [C loss: 2.032600]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 400/1875] [C

[Epoch 0/50] [Batch 531/1875] [C loss: 2.028289]                 [CLF acc: 100% ( 94%)]
[Epoch 0/50] [Batch 532/1875] [C loss: 2.060789]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 533/1875] [C loss: 2.035068]                 [CLF acc:  87% ( 94%)]
[Epoch 0/50] [Batch 534/1875] [C loss: 2.029007]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 535/1875] [C loss: 2.031329]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 536/1875] [C loss: 2.032690]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 537/1875] [C loss: 2.035075]                 [CLF acc:  93% ( 94%)]
[Epoch 0/50] [Batch 538/1875] [C loss: 2.031696]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 539/1875] [C loss: 2.030468]                 [CLF acc:  96% ( 94%)]
[Epoch 0/50] [Batch 540/1875] [C loss: 2.092152]                 [CLF acc:  84% ( 94%)]
[Epoch 0/50] [Batch 541/1875] [C loss: 2.034248]                 [CLF acc:  90% ( 94%)]
[Epoch 0/50] [Batch 542/1875] [C

...

[Epoch 4/50] [Batch 196/1875] [C loss: 2.032107]                 [CLF acc:  96% ( 95%)]
[Epoch 4/50] [Batch 197/1875] [C loss: 2.030713]                 [CLF acc: 100% ( 95%)]
[Epoch 4/50] [Batch 198/1875] [C loss: 2.090467]                 [CLF acc:  87% ( 95%)]
[Epoch 4/50] [Batch 199/1875] [C loss: 2.031137]                 [CLF acc:  96% ( 95%)]
[Epoch 4/50] [Batch 200/1875] [C loss: 2.031950]                 [CLF acc:  93% ( 95%)]
[Epoch 4/50] [Batch 201/1875] [C loss: 2.031292]                 [CLF acc:  93% ( 95%)]
[Epoch 4/50] [Batch 202/1875] [C loss: 2.030625]                 [CLF acc: 100% ( 95%)]
[Epoch 4/50] [Batch 203/1875] [C loss: 2.058690]                 [CLF acc:  93% ( 95%)]
[Epoch 4/50] [Batch 204/1875] [C loss: 2.032123]                 [CLF acc:  96% ( 95%)]
[Epoch 4/50] [Batch 205/1875] [C loss: 2.029135]                 [CLF acc:  96% ( 95%)]
[Epoch 4/50] [Batch 206/1875] [C loss: 2.091313]                 [CLF acc:  78% ( 95%)]
[Epoch 4/50] [Batch 207/1875] [C

Process Process-614:
Process Process-616:
Process Process-618:
Process Process-615:
Process Process-613:
Process Process-617:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()


[Epoch 4/50] [Batch 330/1875] [C loss: 2.030604]                 [CLF acc:  96% ( 94%)]
[Epoch 4/50] [Batch 331/1875] [C loss: 2.030362]                 [CLF acc:  96% ( 94%)]
[Epoch 4/50] [Batch 332/1875] [C loss: 2.069125]                 [CLF acc:  87% ( 94%)]
[Epoch 4/50] [Batch 333/1875] [C loss: 2.033808]                 [CLF acc:  96% ( 94%)]
[Epoch 4/50] [Batch 334/1875] [C loss: 2.032131]                 [CLF acc: 100% ( 94%)]
[Epoch 4/50] [Batch 335/1875] [C loss: 2.061996]                 [CLF acc:  93% ( 94%)]
[Epoch 4/50] [Batch 336/1875] [C loss: 2.030445]                 [CLF acc: 100% ( 94%)]
[Epoch 4/50] [Batch 337/1875] [C loss: 2.059467]                 [CLF acc:  90% ( 94%)]
[Epoch 4/50] [Batch 338/1875] [C loss: 2.031420]                 [CLF acc:  96% ( 94%)]
[Epoch 4/50] [Batch 339/1875] [C loss: 2.032073]                 [CLF acc:  90% ( 94%)]
[Epoch 4/50] [Batch 340/1875] [C loss: 2.063842]                 [CLF acc:  93% ( 94%)]
[Epoch 4/50] [Batch 341/1875] [C

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samp

KeyboardInterrupt: 

## Validate on the original and domain-translated datasets

In [171]:
# turn off model traning/somehow switch to evaluation

simple_classifier.eval()

correct_orignal, correct_MNISTM, correct_translated, total1 = 0,0,0,0

for i, ((imgs_A, labels_A), (imgs_B, labels_B)) in enumerate(zip(usps_data_loader, dataloader_original)):        

        base_tensor = imgs_A.type(FloatTensor) ## .expand(batch_size, 3, img_size, img_size).contiguous()
        imgs_A_g =  Variable(base_tensor)
        imgs_A_c = Variable(base_tensor.view(-1,img_channels*img_size**2))
        
        labels_A = labels_A.type(LongTensor).squeeze()
        imgs_B = Variable(imgs_B.view(-1,img_channels*img_size**2).type(FloatTensor))
        labels_B = labels_B.type(LongTensor).squeeze()

#         z = Variable(FloatTensor(np.random.uniform(-1,1, (batch_size, latent_dimension))))
        z = Variable(FloatTensor(np.random.normal(0,1, (batch_size, latent_dimension)))) 
        original_output = simple_classifier(imgs_A_c)
        MNISTM_output = simple_classifier(imgs_B)
        translated_output = simple_classifier(generator(imgs_A_g, z).view(-1,img_channels*img_size**2))

#             original_loss = task_loss( original_prediction, labels_A)
#             MNISTM_loss = task_loss( MNISTM_prediction, labels_B)
#             translated_loss = task_loss( translated_prediction, labels_A)

        _, predicted = torch.max(original_output.data, 1) # untranslated val (should be lower)
        correct_orignal += (predicted == labels_A).sum()

        _, predicted = torch.max(MNISTM_output.data, 1) # proper val
        correct_MNISTM += (predicted == labels_B).sum()

        _, predicted = torch.max(translated_output.data, 1) # translated val (should be good)
        correct_translated += (predicted == labels_A).sum()

        total1 += labels_A.size(0)
        
        if i%50==0: print(i)

0
50
100
150
200


In [170]:
# lab

In [166]:
total1

7424

In [147]:
# predicted

In [173]:
correct_orignal/total1, correct_MNISTM/total1, correct_translated/total1 # (0.9779, 0.969, 0.9803)

(0.7992995689655172, 0.9451778017241379, 0.8755387931034483)

In [142]:
# (28484, 6990, 29733)

In [ ]:
# produce a classifier plot of accuracy of various letters

In [113]:
# training set: (0.9775833333333334, 0.9827, 0.9805833333333334) 
# (as predicted - classifier_clean is probably overfitting)

In [ ]:
# add some way to evaluate GAN overfitting (Vizualize the latent space before 12)

In [ ]:
# use the below to guide the training on generated samples instead of just randomly selecting them

# class_correct = list(0. for i in range(10))
# class_total = list(0. for i in range(10))
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         outputs = net(images)
#         _, predicted = torch.max(outputs, 1)
#         c = (predicted == labels).squeeze()
#         for i in range(4):
#             label = labels[i]
#             class_correct[label] += c[i].item()
#             class_total[label] += 1


# for i in range(10):
#     print('Accuracy of %5s : %2d %%' % (
#         classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
Results

#1 MNIST and MNIST-M are too similar - domain translation effect is marginal, as mnist itself is an easier task
# hyperparameter tweaking based on paper - outline - it doesn't like training on SVNH
# show closes images on L2 distance from MNIST